In [1]:
# import dependencies
import RMF
import pickle

In [41]:
def has_depth_with_site(root, i):
    """ returns true if node subtree thru first child is at least i
        levels, including the root node itself, and the lead is a site """
#  print root, i, len(root.get_children())
    if (i==1) and root.get_name()=="site":
        return True
    c = root.get_children()
    if len(c) == 0:
        return False
    return has_depth_with_site(c[0], i-1)

def _add_nodes(node, tf, type_prefixes, depth=0):
    '''
    node - rmf node to scan
    tf - typed factory
    type_prefixes - list of full type prefixes (e.g. "Nup1" for "Nup1N")

    adds only nodes whose type name begins with any of the specified type prefixes
    '''
    children = node.get_children()
    ret = []
    #print "inspecting", node.get_name()
    if len(children)==0:
        return ret
    if has_depth_with_site(node, 3) and tf.get_is(children[0]):
        child_type = tf.get(children[0]).get_type_name()
        if any([child_type.startswith(tp) for tp in type_prefixes]):
            ret.append(children)
    for c in children:
        ret += _add_nodes(c, tf,  type_prefixes, depth+1)
    return ret


def load_data(x_coords, y_coords, z_coords, input_rmf_path, min_z_angstrom, rmf_range, is_full_npc=False):
    for rmf_i in rmf_range:
        in_fh = RMF.open_rmf_file_read_only(f"{input_rmf_path}/{rmf_i}.pb.final.rmf")
        rff = RMF.ReferenceFrameFactory(in_fh)
        tf = RMF.TypedFactory(in_fh)
        
        if is_full_npc:
            fg_types = ["Nsp1"] # "Nup100", "Nup116", "Nup159", "Nup49", "Nup57", "Nup145", "Nup1", "Nup60", "Nup2"]
        else:
            fg_types = [f"fg{x}" for x in range(32)]

        # load data
        type2chains={}
        for i, fg_type in enumerate(fg_types):
            type2chains[fg_type] = _add_nodes(in_fh.get_root_node(), tf, [fg_type])
            
        # set frame
        frames = in_fh.get_frames()
        in_fh.set_current_frame(frames[0])

        # read data
        for fg_type in fg_types:
            coord = rff.get(type2chains[fg_type][0][-1]).get_translation() # Get location of terminal bead 
            if coord[2] > min_z_angstrom:
                x_coords.append(coord[0] / 10)
                y_coords.append(coord[1] / 10)
                z_coords.append(coord[2] / 10)
    return

def load_data_multi_sim(input_rmf_path, min_z_angstrom, sim_range, rmf_range, is_full_npc=False):
    x_coords = []
    y_coords = []
    z_coords = []
    
    if sim_range is None: 
        load_data(x_coords, y_coords, z_coords, input_rmf_path, min_z_angstrom, rmf_range, is_full_npc)
    else:
        for sim_i in sim_range:
            # print(sim_i)
            load_data(x_coords, y_coords, z_coords, f"{input_rmf_path}/{sim_i}/", min_z_angstrom, rmf_range, is_full_npc)
            # print(len(x_coords))
    return x_coords, y_coords, z_coords
    

def load_and_save_data(in_path, out_path, sim_range, rmf_range, is_full_npc=False):
    x_coords, y_coords, z_coords = load_data_multi_sim(in_path, 0, sim_range, rmf_range, is_full_npc)
    with open(out_path, "wb") as f:
        pickle.dump([x_coords, y_coords, z_coords], f)
    

In [5]:
x_coords, y_coords, z_coords = load_data("/cs/usr/roi.eliasian/LabFolder/NpcTransportExperiment/HS-AFM-Dataset/dataset/mini_200uM_10ns/", 0)
# x_coords, y_coords, z_coords = load_data("/cs/usr/roi.eliasian/LabFolder/NpcTransportExperiment/HS-AFM-Dataset/dataset/mini_0uM_10ns/")
with open("sr_microscopy_data.pickle", "wb") as f:
    pickle.dump([x_coords, y_coords, z_coords], f)

In [ ]:
in_path_prefix="/cs/usr/roi.eliasian/LabFolder/NpcTransportExperiment/HS-AFM-Dataset/dataset/multi_radii/"
sim_range = [1,2,4,5,6,7,8,9,10] #range(1, 11)
rmf_range = range(100000, 150000, 20000)
load_and_save_data(f"{in_path_prefix}/mini_r185/", "r185_sr.pickle", sim_range, rmf_range)
load_and_save_data(f"{in_path_prefix}/mini_r220/", "r220_sr.pickle", sim_range, rmf_range)
load_and_save_data(f"{in_path_prefix}/mini_r255/", "r255_sr.pickle", sim_range, rmf_range)
load_and_save_data(f"{in_path_prefix}/mini_r295/", "r295_sr.pickle", sim_range, rmf_range)

In [42]:
in_path_prefix="/cs/usr/roi.eliasian/LabFolder/NpcTransportExperiment/HS-AFM-Dataset/dataset/"
sim_range = range(1, 11)
rmf_range = range(100000, 150000, 20000)
load_and_save_data(f"{in_path_prefix}/mini_200uM_100ns", "r185_ntr_sr.pickle", sim_range, rmf_range)

In [19]:
in_path="/cs/usr/roi.eliasian/LabFolder/NpcTransportExperiment/HS-AFM-Dataset/dataset/"
sim_range = None
rmf_range = range(40000, 100000, 20000)
load_and_save_data(f"{in_path_prefix}/full_0uM_1ns/2", "full_sr.pickle", sim_range, rmf_range, is_full_npc=True)